# <b>ETL Process</b>
Use this notebook to complete the ETL Process for each dataset. 

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import re
from datetime import datetime
from langdetect import detect
from googletrans import Translator
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import warnings
import logging

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [112]:
campaign_performance = pd.read_excel('.././Raw Dataset/US Pathway Complete Data.xlsx', sheet_name='campaign_performance')
leads_generated = pd.read_excel('.././Raw Dataset/US Pathway Complete Data.xlsx', sheet_name='Leads Generated')
phone_metrics = pd.read_excel('.././Raw Dataset/US Pathway Complete Data.xlsx', sheet_name='Phone Metrics')
tokens_paid = pd.read_excel('.././Raw Dataset/US Pathway Complete Data.xlsx', sheet_name='Tokens Paid')
candidate_application_tracker = pd.read_excel('.././Raw Dataset/US Pathway Complete Data.xlsx', sheet_name='Candidate Application Tracker')

In [127]:
cp = campaign_performance.copy()
lg = leads_generated.copy()
pm = phone_metrics.copy()
tp = tokens_paid.copy()
cat = candidate_application_tracker.copy()

### <b>Campaign Peformance</b>

#### `Cleaning`
Performing basic operations like : 
- Removing Duplicates 
- Removing Irrelevant Data (URLs, HTML tags)
- Standardizing Capitalization
- Converting Data Types (Dates)
- Clear Formatting (Data Collected from various sources come up with various formats)
- Fixing Errors (US dollars to Rs.)
- Language Translation


In [114]:
def clean_dataframe(df):
    def remove_html_tags(text):
        return re.sub('<.*?>', '', str(text)) if pd.notnull(text) else text

    def standardize_capitalization(text):
        return str(text).lower() if pd.notnull(text) else text

    def convert_date(value):
        if pd.isnull(value):
            return value
        try:
            return pd.to_datetime(value)
        except:
            return value

    def fix_currency(value):
        if pd.isnull(value):
            return value
        if isinstance(value, str) and '$' in value:
            return float(value.replace('$', '')) * 75
        return value

    def safe_strip(x):
        return x.strip() if isinstance(x, str) else x

    df = df.drop_duplicates()

    for column in df.columns:
        if df[column].dtype == 'object':

            df[column] = df[column].apply(lambda x: re.sub(r'http\S+', '', str(x)) if pd.notnull(x) else x)
            df[column] = df[column].apply(remove_html_tags)
            df[column] = df[column].apply(standardize_capitalization)
            df[column] = df[column].apply(fix_currency)

        elif df[column].dtype in ['int64', 'float64']:
            pass

        if df[column].dtype == 'object':
            df[column] = df[column].apply(convert_date)

    df = df.applymap(safe_strip)

    return df

In [115]:
cp = clean_dataframe(cp)

`Identifying Missing Values`

There are three types of missing values:
- MCAR (Missing completely at random)
- MAR (Missing at random)
- MNAR (Missing not at random)


In [6]:
def missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    Percentage = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total, Percentage], axis = 1, keys = ['Total', 'Percentage'])

missing_data(cp)

,Total,Percentage
campaign_start_date,24287,85.116002
adset_name,4247,14.883998
click_through_rate,37,0.129670
dates,0,0.000000
campaign_name,0,0.000000
creative_name,0,0.000000
total_spent,0,0.000000
impressions,0,0.000000
clicks,0,0.000000
leads,0,0.000000


In [7]:
def analyze_missing_data(df, column):
    
    missing_mask = df[column].isnull()
    if not missing_mask.any():
        return "No missing values in this column."
    
    observed = df.loc[:, df.columns != column].groupby(missing_mask).count()
    chi2, p_value, dof, expected = chi2_contingency(observed)
    
    if p_value > 0.05:
        return "MCAR (Missing Completely At Random): The missing data appears to be completely random."
    
    correlations = df.drop(column, axis=1).corrwith(missing_mask)
    if (correlations.abs() > 0.3).any():
        return "MAR (Missing At Random): The missing data appears to be related to other variables."
    
    return "MNAR (Missing Not At Random): The missing data appears to be related to unobserved factors or the missing values themselves."

In [8]:
def analyze_dataframe(df):
    results = {}
    for column in df.columns:
        results[column] = analyze_missing_data(df, column)
    
    result_df = pd.DataFrame.from_dict(results, orient='index', columns=['Missing Data Analysis'])
    return result_df

In [9]:
analyze_dataframe(cp)

,Missing Data Analysis
dates,No missing values in this column.
campaign_name,No missing values in this column.
campaign_start_date,MNAR (Missing Not At Random): The missing data...
creative_name,No missing values in this column.
total_spent,No missing values in this column.
impressions,No missing values in this column.
clicks,No missing values in this column.
click_through_rate,MCAR (Missing Completely At Random): The missi...
leads,No missing values in this column.
platform,No missing values in this column.


In [10]:
cp.drop(columns=['campaign_start_date'], inplace=True)

In [11]:
def impute(df, col):
    df = df.copy()

    le = LabelEncoder()
    df['encoded'] = le.fit_transform(df[col].astype(str))

    imputer = SimpleImputer(strategy='most_frequent')
    df['encoded'] = imputer.fit_transform(df[['encoded']])

    df[col] = le.inverse_transform(df['encoded'])
    df = df.drop('encoded', axis=1)

    return df

cp = impute(cp, 'adset_name')

In [12]:
cp.dropna(inplace=True)

In [13]:
cp.head()

,dates,campaign_name,creative_name,total_spent,impressions,clicks,click_through_rate,leads,platform,adset_name
0,2024-04-29,usp_search_aptntska_250124,--,6766.15,1250,220,0.176000,25.0,google,gmat/gre
1,2024-05-02,usp_2_lead-gen-2_top4-states_050424,#7 ( graphic ) - get a free pass,946.38,3154,9,0.285352,1.0,facebook,usp-2_ka_tn_050424
2,2024-05-02,usp_2_lead-gen-2_top4-states_050424,usp_s3_never-been-easier,170.40,2695,8,0.296846,0.0,facebook,usp-2_ka_tn_050424
3,2024-05-02,usp_2_lead-gen-2_top4-states_050424,usp_s4_do-you-have,55.65,577,2,0.346620,0.0,facebook,usp-2_ka_tn_050424
4,2024-05-02,usp_2_lead-gen-2_top4-states_050424,usp_s1_we-guarantee,296.94,1466,10,0.682128,1.0,facebook,usp-2_ap_ts_050424


In [14]:
cp.to_csv("CampaignPerformance.csv", index=False, header=True)

In [15]:
analyze_dataframe(cp)

,Missing Data Analysis
dates,No missing values in this column.
campaign_name,No missing values in this column.
creative_name,No missing values in this column.
total_spent,No missing values in this column.
impressions,No missing values in this column.
clicks,No missing values in this column.
click_through_rate,No missing values in this column.
leads,No missing values in this column.
platform,No missing values in this column.
adset_name,No missing values in this column.


### <b>Leads Generated</b>

#### `Cleaning`
Performing basic operations like : 
- Removing Duplicates 
- Removing Irrelevant Data (URLs, HTML tags)
- Standardizing Capitalization
- Converting Data Types (Dates)
- Clear Formatting (Data Collected from various sources come up with various formats)
- Fixing Errors (US dollars to Rs.)
- Language Translation


In [116]:
lg = clean_dataframe(lg)

`Identifying Missing Values`

There are three types of missing values:
- MCAR (Missing completely at random)
- MAR (Missing at random)
- MNAR (Missing not at random)


In [117]:
def analyze_dataframe(df):
    results = {}
    for column in df.columns:
        total_missing = df[column].isnull().sum()
        missing_percentage = (total_missing / len(df)) * 100
        missing_type = analyze_missing_data(df, column)
        
        results[column] = {
            'Total Missing': total_missing,
            'Percentage Missing': missing_percentage,
            'Missing Data Type': missing_type
        }
    
    result_df = pd.DataFrame.from_dict(results, orient='index')
    result_df = result_df.sort_values('Percentage Missing', ascending=False)
    
    # Format the percentage column
    result_df['Percentage Missing'] = result_df['Percentage Missing'].apply(lambda x: f"{x:.2f}%")
    
    return result_df

analyze_dataframe(lg)

,Total Missing,Percentage Missing,Missing Data Type
form_name,2339,7.14%,Non-numeric column (7.14% missing)
contacted,1964,5.99%,Non-numeric column (5.99% missing)
lsq_source,765,2.33%,Non-numeric column (2.33% missing)
mapped,711,2.17%,Non-numeric column (2.17% missing)
lsq_lead_stage,672,2.05%,Non-numeric column (2.05% missing)
lsq_lead_owner,607,1.85%,Non-numeric column (1.85% missing)
Ad Name,47,0.14%,Non-numeric column (0.14% missing)
Campaign Name,47,0.14%,Non-numeric column (0.14% missing)
work_experience,7,0.02%,Non-numeric column (0.02% missing)
graduation_degree,6,0.02%,Non-numeric column (0.02% missing)


In [135]:
def impute_column(df, col):
    
    df_copy = df.copy()

    if pd.api.types.is_numeric_dtype(df_copy[col]):
        imputer = KNNImputer(n_neighbors=5)
        df_copy[col] = imputer.fit_transform(df_copy[[col]])
    else:
        le = LabelEncoder()
        nan_mask = df_copy[col].isna()
        le.fit(df_copy[col].dropna())
        df_copy.loc[~nan_mask, col] = le.transform(df_copy.loc[~nan_mask, col])
        imputer = SimpleImputer(strategy='most_frequent')
        df_copy[col] = imputer.fit_transform(df_copy[[col]])
        df_copy[col] = le.inverse_transform(df_copy[col].astype(int))
    
    return df_copy


In [19]:
lg = impute_column(lg, 'contacted')
lg = impute_column(lg, 'lsq_source')
lg = impute_column(lg, 'mapped')
lg = impute_column(lg, 'lsq_lead_stage')
lg = impute_column(lg, 'lsq_lead_owner')
lg = impute_column(lg, 'Ad Name')
lg = impute_column(lg, 'Campaign Name')

In [20]:
lg.dropna(inplace=True)

In [21]:
lg.to_csv("LeadsGenerated.csv", header=True, index=False)

In [22]:
analyze_dataframe(lg)

,Total Missing,Percentage Missing,Missing Data Type
lead_id,0,0.00%,No missing values in this column.
created_time,0,0.00%,No missing values in this column.
Ad Name,0,0.00%,No missing values in this column.
Campaign Name,0,0.00%,No missing values in this column.
form_name,0,0.00%,No missing values in this column.
platform,0,0.00%,No missing values in this column.
lead_id.1,0,0.00%,No missing values in this column.
graduation_degree,0,0.00%,No missing values in this column.
graduation_percentage,0,0.00%,No missing values in this column.
work_experience,0,0.00%,No missing values in this column.


### <b>Phone Metrics</b>

#### `Cleaning`
Performing basic operations like : 
- Removing Duplicates 
- Removing Irrelevant Data (URLs, HTML tags)
- Standardizing Capitalization
- Converting Data Types (Dates)
- Clear Formatting (Data Collected from various sources come up with various formats)
- Fixing Errors (US dollars to Rs.)
- Language Translation


In [128]:
def clean_dataframe(df):
    def remove_html_tags(text):
        return re.sub('<.*?>', '', str(text)) if pd.notnull(text) else text

    def standardize_capitalization(text):
        return str(text).lower() if pd.notnull(text) else text

    def convert_date(value):
        if pd.isnull(value):
            return value
        try:
            return pd.to_datetime(value)
        except:
            return value

    def fix_currency(value):
        if pd.isnull(value):
            return value
        if isinstance(value, str) and '$' in value:
            try:
                cleaned_value = re.sub(r'[^\d.]', '', value)
                return float(cleaned_value) * 75
            except ValueError:
                print(f"Warning: Could not convert '{value}' to float in currency conversion.")
                return value
        return value

    def check_currency_column(df, column):
        problematic_values = df[df[column].apply(lambda x: isinstance(x, str) and '$' in x and not re.sub(r'[^\d.]', '', x).replace('.', '').isdigit())]
        if not problematic_values.empty:
            print(f"Problematic values in {column}:")
            print(problematic_values[column].value_counts())
        return problematic_values
    
    def clean_currency_column(df, column):
        problematic_rows = check_currency_column(df, column)
        if not problematic_rows.empty:
            print(f"Dropping {len(problematic_rows)} rows with invalid currency data.")
            df = df.drop(problematic_rows.index)
        return df
    def safe_strip(x):
        return x.strip() if isinstance(x, str) else x

    df = df.drop_duplicates()

    for column in df.columns:
        if df[column].dtype == 'object':
            if '$' in df[column].values:
                initial_rows = len(df)
                df = clean_currency_column(df, column)
                rows_dropped = initial_rows - len(df)
                if rows_dropped > 0:
                    cleaning_log.append(f"Dropped {rows_dropped} rows from {column} due to invalid currency data.")
            
            df[column] = df[column].apply(fix_currency)

            df[column] = df[column].apply(lambda x: re.sub(r'http\S+', '', str(x)) if pd.notnull(x) else x)
            df[column] = df[column].apply(remove_html_tags)
            df[column] = df[column].apply(standardize_capitalization)
            df[column] = df[column].apply(fix_currency)

        elif df[column].dtype in ['int64', 'float64']:
            pass

        if df[column].dtype == 'object':
            df[column] = df[column].apply(convert_date)

    df = df.applymap(safe_strip)

    return df

In [129]:
pm = clean_dataframe(pm)

`Identifying Missing Values`

There are three types of missing values:
- MCAR (Missing completely at random)
- MAR (Missing at random)
- MNAR (Missing not at random)


In [130]:
def analyze_missing_data(df, column):
    missing_mask = df[column].isnull()
    
    if not missing_mask.any():
        return "No missing values"
    
    missing_percentage = (missing_mask.sum() / len(df)) * 100
    
    if df[column].dtype in ['int64', 'float64']:
        try:
            observed = df.drop(column, axis=1).groupby(missing_mask).count()
            if observed.shape[0] > 1:
                chi2, p_value, dof, expected = chi2_contingency(observed)
                
                if p_value > 0.05:
                    return f"MCAR ({missing_percentage:.2f}% missing)"
                
                correlations = df.drop(column, axis=1).corrwith(missing_mask)
                if (correlations.abs() > 0.3).any():
                    return f"MAR ({missing_percentage:.2f}% missing)"
                
                return f"MNAR ({missing_percentage:.2f}% missing)"
            else:
                return f"Unable to determine ({missing_percentage:.2f}% missing)"
        except ValueError as e:
            print(column, f"Error in analysis: {str(e)} ({missing_percentage:.2f}% missing)")
            return f"Error in analysis: {str(e)} ({missing_percentage:.2f}% missing)"
    else:
        return f"Non-numeric column ({missing_percentage:.2f}% missing)"

def analyze_dataframe(df):
    results = {}
    for column in df.columns:
        results[column] = analyze_missing_data(df, column)
    
    result_df = pd.DataFrame.from_dict(results, orient='index', columns=['Missing Data Analysis'])
    return result_df

analysis_result = analyze_dataframe(pm)
print(analysis_result)

inbound_phone_call_counter Error in analysis: The internally computed table of expected frequencies has a zero element at (0, 13). (23.52% missing)
mobile_number Error in analysis: The internally computed table of expected frequencies has a zero element at (0, 13). (99.57% missing)
outbound_phone_call_counter Error in analysis: The internally computed table of expected frequencies has a zero element at (0, 13). (14.39% missing)
                                                          Missing Data Analysis
lead_number                                                   No missing values
first_name                                   Non-numeric column (0.05% missing)
Lead Id                                     Non-numeric column (25.60% missing)
inbound_phone_call_counter    Error in analysis: The internally computed tab...
mobile_number                 Error in analysis: The internally computed tab...
outbound_phone_call_counter   Error in analysis: The internally computed tab...
lead_sta

In [131]:
pm.drop(columns=['mobile_number','contacted','new_followup','sales_squad','squad_role'], inplace=True)

In [132]:
pm3 = pm.copy()

In [133]:
analysis_result = analyze_dataframe(pm)
print(analysis_result)

                                            Missing Data Analysis
lead_number                                     No missing values
first_name                     Non-numeric column (0.05% missing)
Lead Id                       Non-numeric column (25.60% missing)
inbound_phone_call_counter                  MNAR (23.52% missing)
outbound_phone_call_counter                 MNAR (14.39% missing)
lead_stage                     Non-numeric column (0.08% missing)
lead_score                                      No missing values
owner                                           No missing values
activity                                        No missing values
date_of_call                                    No missing values
status                                          No missing values
call_duration_sec                               No missing values
calls_done_by                                   No missing values
call_time                                       No missing values
calls_done

In [136]:
pm = impute_column(pm, 'inbound_phone_call_counter')
pm = impute_column(pm, 'outbound_phone_call_counter')

In [137]:
pm1 = pm.dropna(subset=['Lead Id'])

In [138]:
pm.drop(columns=['Lead Id'], inplace=True)

In [139]:
pm1.dropna(inplace=True)
pm.dropna(inplace=True)

In [140]:
pm.shape

(257470, 15)

In [141]:
pm1.shape

(191600, 16)

In [142]:
def clean_name(name):
    if not isinstance(name, str):
        return name

    name = name.lower()
    name = re.sub(r'[^a-z\s\'-]', '', name)
    name = re.sub(r'\s+', ' ', name)
    name = ' '.join(word.capitalize() for word in name.split())
    name = re.sub(r'\bMc(\w)', lambda x: 'Mc' + x.group(1).upper(), name)
    name = name.strip()

    return name

In [143]:
pm['first_name'] = pm['first_name'].apply(lambda x: clean_name(x))
pm1['first_name'] = pm1['first_name'].apply(lambda x: clean_name(x))

In [144]:
pm = pm[pm['first_name']!='']
pm1 = pm1[pm1['first_name']!='']

In [146]:
analyze_dataframe(pm1)

,Missing Data Analysis
lead_number,No missing values
first_name,No missing values
Lead Id,No missing values
inbound_phone_call_counter,No missing values
outbound_phone_call_counter,No missing values
lead_stage,No missing values
lead_score,No missing values
owner,No missing values
activity,No missing values
date_of_call,No missing values


In [147]:
pm.to_csv('PhoneMetricsWithoutLeaveId.csv', header=True, index=False)
pm1.to_csv('PhoneMetricsWithLeaveId.csv', header=True, index=False)

### <b>Tokens Paid</b>

#### `Cleaning`
Performing basic operations like : 
- Removing Duplicates 
- Removing Irrelevant Data (URLs, HTML tags)
- Standardizing Capitalization
- Converting Data Types (Dates)
- Clear Formatting (Data Collected from various sources come up with various formats)
- Fixing Errors (US dollars to Rs.)
- Language Translation


In [33]:
tp = clean_dataframe(tp)

`Identifying Missing Values`

There are three types of missing values:
- MCAR (Missing completely at random)
- MAR (Missing at random)
- MNAR (Missing not at random)


In [34]:
tp.drop(columns=['upload_timestamp'], inplace=True)

In [35]:
def analyze_missing_data(df, column):
    
    missing_mask = df[column].isnull()
    if not missing_mask.any():
        return "No missing values in this column."
    
    observed = df.loc[:, df.columns != column].groupby(missing_mask).count()
    chi2, p_value, dof, expected = chi2_contingency(observed)
    
    if p_value > 0.05:
        return "MCAR (Missing Completely At Random): The missing data appears to be completely random."
    
    correlations = df.drop(column, axis=1).corrwith(missing_mask)
    if (correlations.abs() > 0.3).any():
        return "MAR (Missing At Random): The missing data appears to be related to other variables."
    
    return "MNAR (Missing Not At Random): The missing data appears to be related to unobserved factors or the missing values themselves."

def analyze_dataframe(df):
    results = {}
    for column in df.columns:
        total_missing = df[column].isnull().sum()
        missing_percentage = (total_missing / len(df)) * 100
        missing_type = analyze_missing_data(df, column)
        
        results[column] = {
            'Total Missing': total_missing,
            'Percentage Missing': missing_percentage,
            'Missing Data Type': missing_type
        }
    
    result_df = pd.DataFrame.from_dict(results, orient='index')
    result_df = result_df.sort_values('Percentage Missing', ascending=False)
    
    # Format the percentage column
    result_df['Percentage Missing'] = result_df['Percentage Missing'].apply(lambda x: f"{x:.2f}%")
    
    return result_df

analyze_dataframe(tp)

,Total Missing,Percentage Missing,Missing Data Type
Lead ID,146,54.68%,MCAR (Missing Completely At Random): The missi...
status,26,9.74%,MCAR (Missing Completely At Random): The missi...
source,5,1.87%,MCAR (Missing Completely At Random): The missi...
counsellor,2,0.75%,MCAR (Missing Completely At Random): The missi...
prospect_id,2,0.75%,MCAR (Missing Completely At Random): The missi...
created_date,1,0.37%,MCAR (Missing Completely At Random): The missi...
Name,0,0.00%,No missing values in this column.
paid_date,0,0.00%,No missing values in this column.
cohort,0,0.00%,No missing values in this column.
agent,0,0.00%,No missing values in this column.


In [36]:
tp2 = tp.copy()

In [37]:
tp2.drop(columns=['Lead ID'], inplace=True)

In [38]:
analyze_dataframe(tp2)

,Total Missing,Percentage Missing,Missing Data Type
status,26,9.74%,MCAR (Missing Completely At Random): The missi...
source,5,1.87%,MCAR (Missing Completely At Random): The missi...
counsellor,2,0.75%,MCAR (Missing Completely At Random): The missi...
prospect_id,2,0.75%,MCAR (Missing Completely At Random): The missi...
created_date,1,0.37%,MCAR (Missing Completely At Random): The missi...
Name,0,0.00%,No missing values in this column.
paid_date,0,0.00%,No missing values in this column.
cohort,0,0.00%,No missing values in this column.
agent,0,0.00%,No missing values in this column.


In [39]:
tp2['status'] = tp2['status'].fillna("INACTIVE TOKEN")
tp['status'] = tp2['status'].fillna("INACTIVE TOKEN")

In [40]:
tp2.dropna(inplace=True)
tp.dropna(inplace=True)

In [41]:
tp2.to_csv("TokensPaidWithoutLeadID.csv", header=True, index=False)
tp.to_csv("TokensPaidWithLeadID.csv", header=True, index=False)

### <b>Candidate Application Tracker</b>

In [84]:
cat = pd.read_csv("../Raw Dataset/CandidateApplicationTracker.csv")

In [85]:
cat.head(5)

,S. No.,Candidate Name,Counselor Name,Block Amount Received Date,Univ. Pitched,Lead Id,Candidate Preferred College #1,Candidate Preferred College #2,10th Percentage,12th Percentange,UG - CGPA (out of 10),GPA (Out of 4),PG - CGPA or %,Work Exp.,Highest Education,Designation - Company Name,Candidate Application Status,Candidate Loan Status,Remarks - General,Reason for dropping,"If dropped, mention stage of the application",KYC & Acad Documents Received,Tech Exam Scores,Coding Test Score,English Exam Scores,Exam Taking Date,English Interview Score (15),Tech. interview Score (10),Application Type,Intake Status,U1Choice of School #1,U1Choice of Program #1,U1Passport #1,U1Resume #1,U1Degree #1,U1Transcript #1,U1Application Fee #1,U1LOR #1,U1SOP #1,U1Pending Stage #1,U1Application Status #1,U1Application Initation Date #1,U1App Partially Submitted Date #1,U1Application Submission Date #1,U1Offer Letter #1,U1Offer Letter Provision Date #1,U1Deposit #1,U1Deposit Date Paid #1,U2Choice of School #2,U2Choice of Program #2,U2Application Fee #2,U2LOR #2,U2SOP #2,U2Pending Stage #2,U2Application Status #2,U2Application Initation Date #2,U2App Partially Submitted Date #2,U2Application Submission Date #2,U2Offer Letter #2,U2Offer Letter Provision Date #2,U2Deposit #2,U2Deposit Date Paid #2,U3Choice of School #3,U3Choice of Program #3,U3Application Fee #3,U3LOR #3,U3SOP #3,U3Pending Stage #3,U3Application Status #3,U3Application Initation Date #3,U3App Partially Submitted Date #3,U3Application Submission Date #3,U3Offer Letter #3,U3Offer Letter Provision Date #3,U3Deposit #3,U3Deposit Date Paid #3,U3Remarks #3 (If Rejected),U4Choice of School #4,U4Choice of Program #4,U4Application Fee #4,U4LOR #4,U4SOP #4,U4Pending Stage #4,U4Application Status #4,U4Application Initation Date #4,U4App Partially Submitted Date #4,U4Application Submission Date #4,U4Offer Letter #4,U4Offer Letter Provision Date #4,U4Deposit #4,U4Deposit Date Paid #4,U4Remarks #4 (If Rejected),IIM / IIT Status,Loan Status,Loan Partner I,Loan Partner II,Loan Application Submit Date,Existing Loan,CIBIL SCORE,No. of defaults,Aadhar Card,PAN Card,Passport,CV/Resume,CIBIL Score,Bank Statement (Savings - 6mo),Affidavit Letter,Sponsorship Letter,Offer Letter,Offer Letter Submission Date,i20 Financials Submission,i20 Financials Submission Date,i20 Provisioned,i20 Provisioned Date,Remarks,Status,SOP Status
0,1,Vikash,Shailendra Singh,45152,Rutgers,NaN,Rutgers Business School,None,NaN,NaN,7.099,2.83,NaN,3,NaN,NaN,i20 link from University pending,Loan Sanction Letter Pending,NEW i20 to be issued from University,NaN,NaN,True,NR,NR,NR,NR,NR,NR,Deferred,Fall 2024,Rutgers Business School,Master of Information Technology and Analytics,False,False,False,False,True,True,True,Completed,Submitted,NaN,NaN,45178,Received,45188,True,45201,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,False,False,"Application Fee #1, LOR #1, SOP #1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paid,Loan Sanctioned- Futurense,HDFC Credila,MPower,NaN,NaN,NaN,NaN,True,True,True,True,False,False,True,False,False,NaN,True,NaN,Received,14/03/2024,NaN,Defer Fall,NaN
1,2,Hariraj,Shailendra Singh,45188,Fairfield/DePaul,NaN,DePaul Kellstadt Graduate School of Business,None,NaN,NaN,7.95,3.18,NaN,1,NaN,NaN,Deposit Paid & IIM/IIT UnPaid,Loan Sanctioned,NEW i20 to be issued from University,NaN,NaN,True,NR,NR,NR,NR,NR,NR,Deferred,Fall 2024,DePaul Kellstadt Graduate School of Business,MS in Business Analytics,False,False,False,False,True,True,True,Completed,Submitted,NaN,NaN,45195,Received,45204,True,45208,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,False,False,"Application Fee #1, LOR #1, SOP #1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unpaid,Loan Sanctioned- Futurense,HDFC Credila,NaN,NaN,NaN,NaN,NaN,True,True,True,True,True,False,True,False,False,N

#### `Cleaning`
Performing basic operations like : 
- Removing Duplicates 
- Removing Irrelevant Data (URLs, HTML tags)
- Standardizing Capitalization
- Converting Data Types (Dates)
- Clear Formatting (Data Collected from various sources come up with various formats)
- Fixing Errors (US dollars to Rs.)
- Language Translation


In [86]:
cat = clean_dataframe(cat)

`Identifying Missing Values`

There are three types of missing values:
- MCAR (Missing completely at random)
- MAR (Missing at random)
- MNAR (Missing not at random)


In [87]:
cat.head()

,S. No.,Candidate Name,Counselor Name,Block Amount Received Date,Univ. Pitched,Lead Id,Candidate Preferred College #1,Candidate Preferred College #2,10th Percentage,12th Percentange,UG - CGPA (out of 10),GPA (Out of 4),PG - CGPA or %,Work Exp.,Highest Education,Designation - Company Name,Candidate Application Status,Candidate Loan Status,Remarks - General,Reason for dropping,"If dropped, mention stage of the application",KYC & Acad Documents Received,Tech Exam Scores,Coding Test Score,English Exam Scores,Exam Taking Date,English Interview Score (15),Tech. interview Score (10),Application Type,Intake Status,U1Choice of School #1,U1Choice of Program #1,U1Passport #1,U1Resume #1,U1Degree #1,U1Transcript #1,U1Application Fee #1,U1LOR #1,U1SOP #1,U1Pending Stage #1,U1Application Status #1,U1Application Initation Date #1,U1App Partially Submitted Date #1,U1Application Submission Date #1,U1Offer Letter #1,U1Offer Letter Provision Date #1,U1Deposit #1,U1Deposit Date Paid #1,U2Choice of School #2,U2Choice of Program #2,U2Application Fee #2,U2LOR #2,U2SOP #2,U2Pending Stage #2,U2Application Status #2,U2Application Initation Date #2,U2App Partially Submitted Date #2,U2Application Submission Date #2,U2Offer Letter #2,U2Offer Letter Provision Date #2,U2Deposit #2,U2Deposit Date Paid #2,U3Choice of School #3,U3Choice of Program #3,U3Application Fee #3,U3LOR #3,U3SOP #3,U3Pending Stage #3,U3Application Status #3,U3Application Initation Date #3,U3App Partially Submitted Date #3,U3Application Submission Date #3,U3Offer Letter #3,U3Offer Letter Provision Date #3,U3Deposit #3,U3Deposit Date Paid #3,U3Remarks #3 (If Rejected),U4Choice of School #4,U4Choice of Program #4,U4Application Fee #4,U4LOR #4,U4SOP #4,U4Pending Stage #4,U4Application Status #4,U4Application Initation Date #4,U4App Partially Submitted Date #4,U4Application Submission Date #4,U4Offer Letter #4,U4Offer Letter Provision Date #4,U4Deposit #4,U4Deposit Date Paid #4,U4Remarks #4 (If Rejected),IIM / IIT Status,Loan Status,Loan Partner I,Loan Partner II,Loan Application Submit Date,Existing Loan,CIBIL SCORE,No. of defaults,Aadhar Card,PAN Card,Passport,CV/Resume,CIBIL Score,Bank Statement (Savings - 6mo),Affidavit Letter,Sponsorship Letter,Offer Letter,Offer Letter Submission Date,i20 Financials Submission,i20 Financials Submission Date,i20 Provisioned,i20 Provisioned Date,Remarks,Status,SOP Status
0,1,vikash,shailendra singh,45152,rutgers,NaN,rutgers business school,none,NaN,NaN,7.099,2.83,NaN,3,NaN,NaN,i20 link from university pending,loan sanction letter pending,new i20 to be issued from university,NaN,NaN,true,nr,nr,nr,nr,nr,nr,deferred,fall 2024,rutgers business school,master of information technology and analytics,false,false,false,false,true,true,true,completed,submitted,NaN,NaN,45178,received,45188,true,45201,NaN,NaN,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,NaN,NaN,NaN,NaN,false,false,false,"application fee #1, lor #1, sop #1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paid,loan sanctioned- futurense,hdfc credila,mpower,NaN,NaN,NaN,NaN,true,true,true,true,false,false,true,false,false,NaN,true,NaT,received,2024-03-14 00:00:00,NaN,defer fall,NaN
1,2,hariraj,shailendra singh,45188,fairfield/depaul,NaN,depaul kellstadt graduate school of business,none,NaN,NaN,7.95,3.18,NaN,1,NaN,NaN,deposit paid & iim/iit unpaid,loan sanctioned,new i20 to be issued from university,NaN,NaN,true,nr,nr,nr,nr,nr,nr,deferred,fall 2024,depaul kellstadt graduate school of business,ms in business analytics,false,false,false,false,true,true,true,completed,submitted,NaN,NaN,45195,received,45204,true,45208,NaN,NaN,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,NaN,NaN,NaN,false,false,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,false,NaN,NaN,NaN,NaN,false,false,false,"application fee #1, lor #1, sop #1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unpaid,loan sanctioned- futurense,hdfc credila,NaN,NaN,NaN,NaN,NaN,true,true,true,true,true,false,true,fals

`SOP Status` : Statement of Purpose

`No of defaults` : Only one out of 317 rows has a 5+ defaults, potentially an outlier

`U4 Columns` : Potential outliers

In [88]:
cat.drop(columns=['U4Choice of School #4', 'U4Choice of Program #4', 'U4Application Fee #4', 'U4LOR #4', 'U4SOP #4', 'U4Pending Stage #4', 'U4Application Status #4', 'U4Application Initation Date #4', 'U4App Partially Submitted Date #4', 'U4Application Submission Date #4', 'U4Offer Letter #4', 'U4Offer Letter Provision Date #4', 'U4Deposit #4', 'U4Deposit Date Paid #4', 'U4Remarks #4 (If Rejected)'], inplace=True)

In [89]:
cols = ['No. of defaults','Offer Letter Submission Date','i20 Financials Submission Date', 'Remarks','Loan Application Submit Date','If dropped, mention stage of the application','i20 Provisioned Date','Remarks - General']

cat.drop(columns=cols, inplace=True)

In [90]:
cols = [
    'SOP Status', 'Reason for dropping', 'U3Deposit Date Paid #3', 'U2Deposit Date Paid #2', 'U1Deposit Date Paid #1', 'U3App Partially Submitted Date #3', 'U2App Partially Submitted Date #2', 'U1App Partially Submitted Date #1', 'Loan Partner II', 'U3Offer Letter Provision Date #3', 'U2Deposit Date Paid #2', 'U3Application Submission Date #3', 'U3Application Initation Date #3', 'U1Deposit Date Paid #1', 'U2Application Initation Date #2', 'U2Application Submission Date #2', 'U1Application Initation Date #1', 'U1Application Submission Date #1'
]

cols  = list(set(cols))

In [91]:
cat.drop(columns=cols, inplace=True)

In [92]:
cols = ['U3Remarks #3 (If Rejected)','U3Deposit #3','U2Offer Letter Provision Date #2','U1Offer Letter Provision Date #1','Exam Taking Date','CIBIL SCORE','i20 Financials Submission','Offer Letter','Aadhar Card','Bank Statement (Savings - 6mo)','CIBIL Score','PAN Card','Passport','Affidavit Letter','U3Application Fee #3']

cat.drop(columns=cols, inplace=True)

In [93]:
cat['Existing Loan'].fillna(0, inplace=True)
cat['Loan Partner I'].fillna("Nil",inplace=True)
cat['U3Offer Letter #3'].fillna("Awaited",inplace=True)
cat['U3Choice of School #3'].fillna("No Choice",inplace=True)
cat['U3Choice of Program #3'].fillna("No Choice",inplace=True)
cat['U3Application Status #3'].fillna("Not Submitted",inplace=True)
cat['i20 Provisioned'].fillna("Not Received",inplace=True)
cat['PG - CGPA or %'].fillna(0,inplace=True)
cat['English Interview Score (15)'].fillna(0,inplace=True)
cat['Tech. interview Score (10)'].fillna(0,inplace=True)
cat['Designation - Company Name '].fillna("No company",inplace=True)
cat['12th Percentange'].fillna(0,inplace=True)
cat['10th Percentage '].fillna(0,inplace=True)
cat['U2Offer Letter #2'].fillna("Awaited",inplace=True)
cat['Highest Education'].fillna("No Higher Education",inplace=True)
cat['English Exam Scores'].fillna(0,inplace=True)
cat['Coding Test Score'].fillna(0,inplace=True)
cat['Tech Exam Scores'].fillna(0,inplace=True)
cat['U2Application Status #2'].fillna("Not Submitted",inplace=True)
cat['U2Choice of Program #2'].fillna("No Choice",inplace=True)
cat['U2Choice of School #2'].fillna("No choice",inplace=True)
cat['U1Offer Letter #1'].fillna("Not Received",inplace=True)
cat['Univ. Pitched'].fillna("Not Pitched",inplace=True)
cat['Candidate Preferred College #2'].fillna("No Preference",inplace=True)
cat['Application Type'].fillna('New',inplace=True)
cat['U1Application Status #1'].fillna("Not Submitted",inplace=True)
cat['U1Choice of School #1'].fillna("No Choice",inplace=True)
cat['U1Choice of Program #1'].fillna("No Choice",inplace=True)
cat['Work Exp.'].fillna(0,inplace=True)
cat['CV/Resume'].fillna(cat['CV/Resume'].mode(),inplace=True)
cat['Sponsorship Letter'].fillna(cat['Sponsorship Letter'].mode(), inplace=True)



In [94]:
cat['CV/Resume'].fillna(cat['CV/Resume'].mode(),inplace=True)
cat['Sponsorship Letter'].fillna(cat['Sponsorship Letter'].mode(),inplace=True)
cat['Candidate Preferred College #1'].fillna('No Preference',inplace=True)
cat['U1Deposit #1'].fillna(cat['U1Deposit #1'].mode(), inplace=True)


In [95]:
cols =['U3SOP #3','U3LOR #3','U3Pending Stage #3','U1Deposit #1']

cat.drop(columns=cols, inplace=True)

In [96]:
cols = ['U2LOR #2','U2SOP #2','U1LOR #1','U1SOP #1','U2Pending Stage #2','U1Pending Stage #1','U2Pending Stage #2','U2Application Fee #2','U1Application Fee #1','U1Pending Stage #1']

cat.drop(columns=cols, inplace=True)

In [97]:
def missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    Percentage = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
    return pd.concat([total, Percentage], axis = 1, keys = ['Total', 'Percentage'])

missing_data(cat)

,Total,Percentage
Lead Id,186,59.047619
GPA (Out of 4),35,11.111111
UG - CGPA (out of 10),34,10.793651
Sponsorship Letter,30,9.523810
CV/Resume,30,9.523810
U2Deposit #2,12,3.809524
U1Passport #1,5,1.587302
U1Resume #1,5,1.587302
U1Transcript #1,5,1.587302
U1Degree #1,5,1.587302


In [98]:
cat2 = cat.dropna(subset=['Lead Id'])
cat.drop(columns=['Lead Id'], inplace=True)

In [99]:
for column in cat.columns:
    mode_value = cat[column].mode()[0]
    cat[column].fillna(mode_value, inplace=True)

In [100]:
cat.shape

(315, 50)

In [101]:
for column in cat2.columns:
    mode_value = cat2[column].mode()[0]
    cat2[column].fillna(mode_value, inplace=True)

In [111]:
cat.to_csv("CandidateApplicationTrackerWithoutLeadIDs.csv", header=True, index=False)
cat2.to_csv("CandidateApplicationTrackerWithLeadIDs.csv", header=True, index=False)